# Gathering Data

The data on the website are not in one place, but spread out through several pages on the 
Southeast regional Climate Center website. Thus, my first step was not within python but gathering
the data and placing it into a manageable format.

North Carolina is separated into 3 regions Morehead City, Raleigh/Durham, and Wilmington. However, all of the weather stations that the SRCC uses in North Carolina are not all contained in these 3 regions. Several other regions that are placed under other states have regions that overlap into North Carolina, thus one must click on each of the regions within NC and all of the neighboring states regions in order to gather the data for all of the stations that are located in North Carolina. Then I had to go through and click on each location within the region, select the appropriate monthly rain data and copy and paste this data into an excel file as an individual sheet.

In addition, my excel data file contains surrounding locations from these neighboring states that are close to North Carolina. We will use these points as exogenous datapoints to see any relationship between these locations and the locations within NC 

# Data Wrangling

#### Step 1

I imported the necessary modules and then load the excel spreadsheet that I used to collect all of the data from the website.

In [63]:
import pandas as pd
import math
import numpy as np
file = 'Data/NC Monthly Precipitation Data.xlsx'
NCdata = pd.ExcelFile(file)

#### Step 2
I needed to bring all of the separate sheets in excel together into a single dataframe. Thus, this function uses a for loop to parse out each sheet from the excel file. Once I had the sheet, all of the months were in separate columns. The function takes all of the months and places them next to the year then places the rainfall amounts for each month in the next column. The function merges the resulting dataframe into the blank dataframe on the year and the month columns with an outer join in order to catch all the data from both the original and merging dataframe. 

In [64]:
blank = pd.DataFrame()
def datachunks(s, e, df): #s and e stand for the beginning and end of the chunk you want
    for i in range(s,e):
        to_merge = NCdata.parse(i, skiprows=[0,1], usecols=[0,1,2,3,4,5,6,7,8,9,10,11,12]) #first two rows in the data were titles
        to_merge = to_merge.dropna() #removes two rows from the data that were labeled as NaN and not needed
        to_merge = to_merge.set_index('Year') #set the index to year to remove the following 3 rows
        to_merge = to_merge.drop(['Mean','Max', 'Min'])
        to_merge = to_merge.reset_index() #resets the index so that the dataframe can be melted on Year
        to_merge1 = pd.melt(to_merge, id_vars=['Year'], value_vars=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug',
        'Sep','Oct','Nov','Dec'], var_name='Month') 
        to_merge1.iloc[:,2] = pd.to_numeric(to_merge1.iloc[:,2], errors = 'coerce')
        if i == 0:
            df = to_merge1
        else:
            df = pd.merge(df, to_merge1, on = ['Year','Month'], how = 'outer')
    return df

ncdata = datachunks(0,234, blank)
ncdata.tail()

,Year,Month,value_x,value_y,value_x,value_y,value_x,value_y,value_x,value_y,...,value_x,value_y,value_x,value_y,value_x,value_y,value_x,value_y,value_x,value_y
1951,1864,Dec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1952,1865,Dec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1953,1866,Dec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1954,1867,Dec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1955,1868,Dec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Step 3
Column names to be placed on top of the dataframe

In [65]:
colnames = ['Year', 'Month']
names = NCdata.sheet_names
ncdata.columns = colnames + names
ncdata.head()

,Year,Month,"Raleigh, NC","Greensboro, NC","Fayetteville, NC","Albemarle, NC","Arcola, NC","Asheboro, NC","Burlington, NC","Carthage, NC",...,"ROGERSVILLE 1 NE, TN","SODDY DAISY-MOWBRAY MTN, TN","SPRING CITY, TN","TAZEWELL, TN","TOWNSEND 5S, TN","KING, NC","ABINGDON 3S, VA","WISE 1SE, VA","John Kerr Dam, VA","Emporia, VA"
0,1887,Jan,3.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1888,Jan,3.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1889,Jan,6.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1890,Jan,0.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1891,Jan,3.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Step 4
In order to sort the data based on the year and month I needed to first convert the columns to datetime

In [66]:
ncdata["Month"] = pd.to_datetime(ncdata.Month, format='%b', errors='coerce').dt.month
ncdata["Year"] = pd.to_datetime(ncdata.Year, format='%Y', errors='coerce').dt.year
print(ncdata[['Year', 'Month']].head())


   Year  Month
0  1887      1
1  1888      1
2  1889      1
3  1890      1
4  1891      1


In [67]:
#this sorts the data based on year then month
ncdata = ncdata.sort_values(['Year','Month'])
ncdata.tail(12)

,Year,Month,"Raleigh, NC","Greensboro, NC","Fayetteville, NC","Albemarle, NC","Arcola, NC","Asheboro, NC","Burlington, NC","Carthage, NC",...,"ROGERSVILLE 1 NE, TN","SODDY DAISY-MOWBRAY MTN, TN","SPRING CITY, TN","TAZEWELL, TN","TOWNSEND 5S, TN","KING, NC","ABINGDON 3S, VA","WISE 1SE, VA","John Kerr Dam, VA","Emporia, VA"
132,2019,1,3.43,4.01,2.77,4.60,NaN,4.43,NaN,NaN,...,4.62,7.10,8.27,5.54,5.32,4.65,5.10,4.17,3.38,NaN
265,2019,2,4.26,5.89,2.05,2.97,NaN,5.86,NaN,4.67,...,12.22,10.00,14.42,14.67,9.64,6.00,9.22,10.51,5.70,5.61
398,2019,3,3.17,3.19,2.66,2.62,NaN,3.86,NaN,NaN,...,4.74,4.77,6.98,4.94,5.51,2.35,3.13,3.78,3.63,3.90
531,2019,4,6.36,4.16,4.89,6.60,NaN,5.42,NaN,NaN,...,5.55,7.75,6.29,3.58,7.97,4.39,4.52,5.28,3.23,3.98
664,2019,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.82,NaN,4.34,5.10,5.47,2.22,2.82,4.39,NaN,1.23
797,2019,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
930,2019,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1063,2019,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1196,2019,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1329,2019,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Step 5
Instead of separate columns for year and month the following code creates a single Date column and sets it as the index. The index is a string because datetime does not allow for dates without a day; however, having a day listed in the datetime would not be reasonable in this dataset because these are monthly totals of rainfall not occurring on a single day. 

In [68]:
ncdata['Year'] = ncdata.Year.apply(str)
ncdata['Month'] = ncdata.Month.apply(str)
ncdata['Date (month-year)'] = ncdata['Month'] + '-' + ncdata['Year']
ncdata1 = ncdata.set_index('Date (month-year)')
ncdata1 = ncdata1.drop(['Year', 'Month'], axis=1)
ncdata1.head()

,"Raleigh, NC","Greensboro, NC","Fayetteville, NC","Albemarle, NC","Arcola, NC","Asheboro, NC","Burlington, NC","Carthage, NC","Chapel Hill, NC","Clayton, NC",...,"ROGERSVILLE 1 NE, TN","SODDY DAISY-MOWBRAY MTN, TN","SPRING CITY, TN","TAZEWELL, TN","TOWNSEND 5S, TN","KING, NC","ABINGDON 3S, VA","WISE 1SE, VA","John Kerr Dam, VA","Emporia, VA"
Date (month-year),,,,,,,,,,,,,,,,,,,,,
1-1857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2-1857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3-1857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4-1857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5-1857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Step 6 - removing impossible data
I gathered this data in May 2019; thus, it was impossible to have any totals from months that had not happened yet; therefore, I removed them

In [69]:
ncdata1 = ncdata1.drop(['5-2019', '6-2019', '7-2019', '8-2019','9-2019','10-2019','11-2019','12-2019'], axis=0)
ncdata1.tail(12)

,"Raleigh, NC","Greensboro, NC","Fayetteville, NC","Albemarle, NC","Arcola, NC","Asheboro, NC","Burlington, NC","Carthage, NC","Chapel Hill, NC","Clayton, NC",...,"ROGERSVILLE 1 NE, TN","SODDY DAISY-MOWBRAY MTN, TN","SPRING CITY, TN","TAZEWELL, TN","TOWNSEND 5S, TN","KING, NC","ABINGDON 3S, VA","WISE 1SE, VA","John Kerr Dam, VA","Emporia, VA"
Date (month-year),,,,,,,,,,,,,,,,,,,,,
5-2018,5.12,3.77,2.68,3.37,NaN,7.56,NaN,2.98,NaN,4.05,...,5.23,7.40,3.55,5.94,4.27,9.85,4.10,5.39,8.03,3.55
6-2018,1.98,2.53,4.31,3.11,NaN,1.79,NaN,NaN,NaN,3.55,...,5.26,7.35,9.26,7.22,5.27,2.74,3.39,6.51,5.82,5.03
7-2018,4.92,6.04,6.36,5.83,NaN,2.69,NaN,NaN,NaN,7.25,...,2.13,3.35,7.39,3.10,4.19,2.29,3.37,5.08,5.63,5.23
8-2018,6.62,6.97,5.75,5.37,NaN,9.26,NaN,9.43,NaN,7.36,...,4.85,6.15,7.39,5.51,6.98,6.72,9.23,3.46,4.64,6.39
9-2018,7.97,9.17,15.54,12.83,NaN,14.11,NaN,NaN,NaN,10.65,...,7.18,11.25,11.61,4.61,7.61,7.00,7.03,6.70,4.70,4.98
10-2018,4.58,7.22,2.22,6.87,NaN,5.24,NaN,NaN,NaN,3.93,...,4.05,2.80,2.89,2.63,4.18,7.40,3.58,3.30,5.05,4.49
11-2018,7.10,6.46,5.31,8.22,NaN,9.18,NaN,NaN,NaN,6.29,...,5.88,8.10,7.62,5.17,NaN,5.83,5.25,5.21,6.61,6.29
12-2018,6.23,6.99,6.00,8.43,NaN,6.41,NaN,NaN,NaN,9.24,...,6.72,7.11,9.49,7.04,8.75,6.54,5.12,6.53,4.85,4.44
1-2019,3.43,4.01,2.77,4.60,NaN,4.43,NaN,NaN,NaN,4.74,...,4.62,7.10,8.27,5.54,5.32,4.65,5.10,4.17,3.38,NaN


#### Step 7
There was a lot of missing data from several locations. Due to this I created the following for loop in order to see which rows (corresponding to a single month) had at least 70% of data. Since the function len() counts missing data while the method .count() does not, I used these two functions to figure out the percentage that each row has and made it a column in the dataframe called 'percent_number'

I found that from January 1956-present all had data from at least 70% of the locations. 

In [70]:
lop = []
for i in ncdata1.index:
    l = len(ncdata1.loc[i])
    c = ncdata1.loc[i].count()
    percent = (c/l)*100
    if i == 0:
        lop = [percent]
    else:
        lop = lop + [percent]
ncdata1['percent_number'] = lop
ncdata1.percent_number[ncdata1.percent_number >= 70].head(20)

Date (month-year)
2-1956     70.512821
3-1956     70.940171
4-1956     72.649573
5-1956     71.794872
6-1956     72.649573
7-1956     71.367521
8-1956     70.940171
9-1956     71.367521
10-1956    71.367521
11-1956    71.794872
12-1956    72.222222
1-1957     70.085470
2-1957     71.367521
3-1957     72.222222
4-1957     72.649573
5-1957     71.794872
6-1957     72.222222
7-1957     72.222222
8-1957     72.649573
9-1957     72.649573
Name: percent_number, dtype: float64

#### Step 8
Since I had the Date column as my index and I didn't want to remove it, I created a new index row called row_number. I used the row number to figure out which row 1-1956 was located at in order to create the dataframe that includes only data from 1-1956-present

In [74]:
nl = [i for i in range(1948)]
ncdata1['row_number'] = nl 
ncdata1.row_number.loc['1-1956'] # provides the row which Jan 1956 is located

1188

In [75]:
ncdata1 = ncdata1[ncdata1.row_number >= 1188]
ncdata1.head()

,"Raleigh, NC","Greensboro, NC","Fayetteville, NC","Albemarle, NC","Arcola, NC","Asheboro, NC","Burlington, NC","Carthage, NC","Chapel Hill, NC","Clayton, NC",...,"SPRING CITY, TN","TAZEWELL, TN","TOWNSEND 5S, TN","KING, NC","ABINGDON 3S, VA","WISE 1SE, VA","John Kerr Dam, VA","Emporia, VA",percent_number,numbered
Date (month-year),,,,,,,,,,,,,,,,,,,,,
1-1956,1.05,1.04,1.89,1.43,NaN,1.23,1.51,1.68,1.46,1.11,...,NaN,NaN,NaN,NaN,NaN,3.17,0.74,1.25,68.803419,1188
2-1956,4.70,5.32,6.66,5.65,4.82,5.57,5.92,5.09,5.66,6.15,...,NaN,NaN,NaN,NaN,NaN,8.93,5.32,4.04,70.512821,1189
3-1956,3.37,2.54,2.82,3.84,3.73,3.49,3.03,3.89,3.70,4.10,...,NaN,NaN,NaN,NaN,NaN,5.96,3.59,2.69,70.940171,1190
4-1956,4.74,3.69,2.11,4.51,4.10,2.75,3.53,3.21,2.97,3.78,...,NaN,NaN,NaN,NaN,NaN,6.48,2.71,3.07,72.649573,1191
5-1956,1.88,4.18,2.43,1.71,2.45,2.81,2.22,3.41,3.35,3.73,...,NaN,NaN,NaN,NaN,NaN,4.18,2.56,2.02,71.794872,1192


In [76]:
#this drops the percent number column that is no longer needed.
ncdata1 = ncdata1.drop(['percent_number'],axis=1)

#### Step 9
The dataset also has some missing data from when the location began gathering data to the present. For example, even though Raleigh, NC has been gathering data from 1-1956 until the present there was one month in October of 2000 where the monthly total was not recorded. Thus the following function fills in missing data that are contained within the locations. This function finds the months with missing data and fills them in by averaging the rainfall totals from the previous year, previous month and next month. If the any of these points are not available either the function uses the data from two years prior, two months prior, or two months after. If just one of these is not available it ignores the NaN and averages the other two, otherwise it keeps the datapoint as NaN. Thus, this function does not get rid of all missing values, but fills in the missing values as long as there is adequate data to do so.

In [77]:
def missingfill(df, column):
    missing = df.index[df[column].isnull()]
    if len(missing) > 0:
        for n in missing:
            moth = df.loc[n].row_number # finds the row index for the missing data point
            if ((moth >= 1200) & (moth <= 1945)): #must be a year after 1-1956 (rownumber=1188) otherwise it is impossible to have the previous year's data to gather from
                ly = moth - 12 #the previous year's row number
                lyrd = df[[column]][df['row_number'] == ly] # the previous year's rainfall amount as a dataframe
                lyrd1 = lyrd[column][0] #separates the value of the previous year's dataframe to just the rainfall amount
                lm = moth - 1 # the next 6 lines perform the same as the previous 3 except for previous month and following month
                lmrd = df[[column]][df['row_number'] == lm]
                lmrd1 = lmrd[column][0]
                nm = moth + 1
                nmrd = df[[column]][df['row_number'] == nm]
                nmrd1 = nmrd[column][0]
                if ((math.isnan(lyrd1)) & (moth >= 1212)): # if the previous year was not available, go back 2 years
                    twy = moth - 24
                    twyrd = df[[column]][df['row_number'] == twy]
                    lyrd1 = twyrd[column][0]
                if (math.isnan(lmrd1)): #if the previous month was not available, go back 2 months
                    lm = moth - 2
                    lmrd = df[[column]][df['row_number'] == lm]
                    lmrd1 = lmrd[column][0]
                if (math.isnan(nmrd1)): #if the next month was not available, go forward 2 months
                    nm = moth + 2
                    nmrd = df[[column]][df['row_number'] == nm]
                    nmrd1 = nmrd[column][0]
                newpoint = np.nanmean([lyrd1,lmrd1,nmrd1]) #finds the average of the 3 values 
                df.loc[n,column] = newpoint #places the value into the missing data slot
    return(df)


#### Step 10 
performs the function defined in the previous cell and applies the function to every column. The runtime warning just means that np.nanmean has only nan values and thus is returning a nan value again, which is fine. 

In [79]:
for i in ncdata1.columns:
    ncdata1 = missingfill(ncdata1, i)
ncdata1.info()
    


C:\Users\Billy\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: Mean of empty slice


<class 'pandas.core.frame.DataFrame'>
Index: 760 entries, 1-1956 to 4-2019
Columns: 235 entries, Raleigh, NC to numbered
dtypes: float64(234), int64(1)
memory usage: 1.4+ MB


#### Step 11 - Final step for Data Wrangling

Drop the row_number column since it is no longer needed, and make a csv file from the dataframe. 

In [ ]:
# drops the row_number column since it is no longer needed. 
ncdata1 = ncdata1.drop(['row_number'],axis=1)
ncdata1.to_csv('ncrainfalldata.csv')